# Sunfounder Kit Lesson 1: Dual-color LED

This module board contains an LED light that can be set to one of two different colors: red or green.

## The setup
The latest module (the little breadboard) differs slightly from the wiring diagram- GND is last instead of in the middle. Make sure that the yellow wire goes to the R pin spot on the breadboard, the red wire to the G pin spot, and the black wire to GND.

<img src=files/01_dual_color_led_bb.png width=650>




## Code to blink between colors
When you run the code below, the light should blink between red and green until you stop it using the black square icon above.

In [1]:
"""
Blinking red-green light code written by Libby Shoop.
Sunfounder code was fairly convoluted so I developed this to make it a bit simpler to follow.
"""
import time
import RPi.GPIO as GPIO      #We use this module to connect input and output devices to the GPIO pins

############ global variables with needed initial setup  ########
GPIO.setmode(GPIO.BOARD)
GREEN_PIN = 12
RED_PIN = 11

GPIO.setup(GREEN_PIN, GPIO.OUT)   # set pin 12 as output for green led  
GPIO.setup(RED_PIN, GPIO.OUT)     # set pin 11 as output for red led  

# PWM is for pulse wave modulator.
# We simulate the light staying on by flashing it really quickly (here 100 hz)
# using this PWM 'object' in the GPIO module.
green = GPIO.PWM(GREEN_PIN, 100)    # create object green for PWM on port 12 at 100 Hertz  
red = GPIO.PWM(RED_PIN, 100)        # create object red for PWM on port 11 at 100 Hertz  

###### end global variables   ###################################

"""
To start, red will be set to on, and green will be set to off
"""
def startup():
    green.start(0)              # start green led on 0 percent duty cycle (off)  
    red.start(100)              # red fully on (100%)  

"""
Set the light red for three seconds, then green for three seconds
"""
def red_then_green():
    green.ChangeDutyCycle(0)  # green off
    red.ChangeDutyCycle(60)   # red on. because the red is so much brighter than green, we tone it down
    time.sleep(0.5)
    red.ChangeDutyCycle(0)     # red off
    green.ChangeDutyCycle(100) # green on full strength
    time.sleep(0.5)

"""
Main program:
now we will have one on and one off, and flip them on and off, waiting 3 seconds in between
"""
################################################
if __name__ == "__main__":
    startup()
    try:  
        while True:             # loop repeatedly, waiting for interrupt when black box is pressed
            red_then_green()

    except KeyboardInterrupt:  # get here when stop button  in notebook is pressed
        green.stop()            # stop the green PWM output  
        red.stop()              # stop the red PWM output  
        GPIO.cleanup()          # clean up GPIO on CTRL+C exit  
####################################################################### end main

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:13: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:14: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.


## Code details

Hit escape and then the l key to show line numbers in the code above. Look at line 9. As we study these modules in depth and keep studying python, this will become clearer. For now, recall the picture from the getting started notebook of the T 'cobbler' that we have connected to the breadboard:

<img src=files/pinout_labels.jpg width=550>

Notice that the pin on the T cobbler that is labeled GPIO17 is alternately labeled 11 in the above picture. Notice that the pin on the T cobbler that is labeled GPIO18 is alternately labeled 12 in the above picture. Also notice that you plugged the red and green pins from the LED module into these two pin locations on the board.

We will learn more about this as we keep working with these modules. For now, note that line 9 is setting the code to use the numbering on the white dots in the picture above, not the numbering on the T cobbler. See [this post](http://raspberrypi.stackexchange.com/questions/12966/what-is-the-difference-between-board-and-bcm-for-gpio-pin-numbering) for an explanation of this. Then lines 10 and 11 are setting constant variables for the red and green pins using that numbering.


## Change the timing of the blinks

You can experiment with changing the amount of time before the color changes by updating the number of seconds given to the function called time.sleep().



## Changing the brightness of either color

This code is a variation on the above theme of blinking on and off, where instead we see how we can con tinuously change the brightness of the LED light. This serves to give you an idea of the range of brightness of each color. Red can glow considerably brighter than green.

In [ ]:
"""
The gist of this code came from:
https://sourceforge.net/p/raspberry-gpio-python/wiki/PWM/

Edited for clarity, modularity,  and to use both colors by Libby Shoop
"""
import time
import RPi.GPIO as GPIO

GPIO.setmode(GPIO.BOARD)
green = 12
red = 11

"""
Given a color pin number using GPIO.BOARD numbering setup,
this function will first quickly brighten the LED from the minimum
to the maximun brightness, changing every 0.1 seconds, then turn 
around and dim it back down to the minimum again in 0.1 second steps.
"""
def brighten_dim(color):
    GPIO.setup(color, GPIO.OUT)     #color pin is the output

    p = GPIO.PWM(color, 50)  # channel=color frequency=50Hz
    p.start(0)
  
    for dc in range(0, 101, 5):
        p.ChangeDutyCycle(dc)
        time.sleep(0.1)
    for dc in range(100, -1, -5):
        p.ChangeDutyCycle(dc)
        time.sleep(0.1)
    return p

"""
This main function first brightens and dims the gren light repeatedly,
waiting for the black button to be pressed to interrupt and go on to the next phase,
where the red light will be brightened and dimmed repeatedly until the black
button is pressed to stop the light completely.
"""
if __name__ == "__main__":
    try:
        while True:
            g_pwm = brighten_dim(green)
    except KeyboardInterrupt:   #press stop button to go on to red light
        g_pwm.stop()
    
    try:
        while True:
            r_pwm = brighten_dim(red)
    except KeyboardInterrupt:
        r_pwm.stop()

## when stop button pressed second time, turn all off

GPIO.cleanup()

### Want to know more?
If you execute the next cell, you can get some information about what some of the GPIO library functions are doing and what else is available. If you don't want to see it anymore, you can clear the cell using the Cell menu in the notebook menu bar.

In [1]:
import RPi.GPIO as GPIO

help(GPIO)

Help on package RPi.GPIO in RPi:

NAME
    RPi.GPIO - Copyright (c) 2012-2016 Ben Croston

FILE
    /usr/lib/python2.7/dist-packages/RPi/GPIO/__init__.py

DESCRIPTION
    Permission is hereby granted, free of charge, to any person obtaining a copy of
    this software and associated documentation files (the "Software"), to deal in
    the Software without restriction, including without limitation the rights to
    use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies
    of the Software, and to permit persons to whom the Software is furnished to do
    so, subject to the following conditions:
    
    The above copyright notice and this permission notice shall be included in all
    copies or substantial portions of the Software.
    
    THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
    IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
    FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL

## Back and forth

Now we can see how we can go back and forth between red and green, brightening and dimming together for an interesting effect.

In [ ]:
"""
The gist of this code came from :
http://raspi.tv/2013/how-to-use-soft-pwm-in-rpi-gpio-pt-2-led-dimming-and-motor-speed-control
"""
import time
import RPi.GPIO as GPIO

# global variables
GPIO.setmode(GPIO.BOARD)
GREEN_PIN = 12
RED_PIN = 11

GPIO.setup(GREEN_PIN, GPIO.OUT)# set pin 12 as output for green led  
GPIO.setup(RED_PIN, GPIO.OUT)# set pin 11 as output for red led  

# PWM is pulse wave modulator
# We simulate the light staying on by flashing it really quickly (here 100 hz)
green = GPIO.PWM(GREEN_PIN, 100)    # create object green for PWM on port 12 at 100 Hertz  
red = GPIO.PWM(RED_PIN, 100)      # create object red for PWM on port 11 at 100 Hertz  

###### end global variables   ###################################

green.start(0)              # start green led on 0 percent duty cycle (off)  
red.start(100)              # red fully on (100%)  
  
# now the fun starts, we'll vary the duty cycle to   
# dim/brighten the leds, so one is bright while the other is dim 
# The duty cycle is a way to set the brightness of the LED : 0 is off and 100 is a bright as it gets
################################################
if __name__ == "__main__":  
    pause_time = 0.02           # you can change this to slow down/speed up the ramp up of color  

    try:  
        while True:  # keep repeating until stop button in notebook is pressed
            for i in range(0,101):      # 101 because it stops when it finishes 100  
                green.ChangeDutyCycle(i)  
                red.ChangeDutyCycle(100 - i)  
                time.sleep(pause_time) 
            time.sleep(2)
            for i in range(100,-1,-1):      # from 100 to zero in steps of -1  
                green.ChangeDutyCycle(i)  
                red.ChangeDutyCycle(100 - i)  
                time.sleep(pause_time) 
            time.sleep(1)
    except KeyboardInterrupt:  # get here when stop button in notebook is pressed
        green.stop()            # stop the green PWM output  
        red.stop()              # stop the red PWM output  
        GPIO.cleanup()          # clean up GPIO on CTRL+C exit  

## Cleaning up for the next one

Once you are done experimenting with this module, do the following:

- Remove the 3 pins for the LED module from the board.
- Leave the other 4 wires near the top of the board.
- Go to the tab labeled Home in this browser and chose the square button to the right of this notebook file. Thhen pick the orange shutdown option.
- Go back to this window and close it

## The code that Sunfounder provided
The following code code seems more complicated than it needs to be. I include it here for completeness and since some of the other modules we try from Sunfounder may have code like this. It was what they provided for the blinking light.

In [6]:
#!/usr/bin/env python
import RPi.GPIO as GPIO
import time


# global variables used in the code
colors = [0xFF00, 0x00FF]   #0xFF00 is the setting for red
                            #0x00FF is the setting for green
    
pins = {'pin_R':11, 'pin_G':12}  # pins is a dict

p_R = GPIO.PWM(pins['pin_R'], 2000)  # set Frequecy to 2KHz
p_G = GPIO.PWM(pins['pin_G'], 2000)


def setup():
    GPIO.setmode(GPIO.BOARD)       # Numbers GPIOs by physical location

    for i in pins:
        GPIO.setup(pins[i], GPIO.OUT)   # Set pins' mode is output
        GPIO.output(pins[i], GPIO.HIGH) # Set pins to high(+3.3V) to off led


    p_R.start(0)      # Initial duty Cycle = 0(leds off)
    p_G.start(0)

def map(x, in_min, in_max, out_min, out_max):
	return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

def setColor(col):   # For example : col = 0x112233
	R_val = (col & 0x1100) >> 8
	G_val = (col & 0x0011) >> 0
	
	R_val = map(R_val, 0, 255, 0, 100)
	G_val = map(G_val, 0, 255, 0, 100)
	
	p_R.ChangeDutyCycle(R_val)     # Change duty cycle
	p_G.ChangeDutyCycle(G_val)

def loop():
    while True:
        for col in colors:
            setColor(col)
            time.sleep(2) #wait 2 seconds

def destroy():
	p_R.stop()
	p_G.stop()
	for i in pins:
		GPIO.output(pins[i], GPIO.HIGH)    # Turn off all leds
	GPIO.cleanup()




In [7]:
if __name__ == "__main__":
    setup()
    try:
        loop()
    except KeyboardInterrupt:
        destroy()